#### This code will consolidate all the logs from DataDog, CAL and FPTI and save it in another table in chronological order . E.g. this consolidates the logs for 2025-01-15 from 9 to 10 am.

In [1]:
import pandas as pd

df = None
# Load CSV files with proper datetime parsing
def consolidate(start_time, end_time):
    global df
    df_dd = pd.read_csv("datadog_mobile_logs.csv", parse_dates=["timestamp"])
    df_be = pd.read_csv("backend_logs.csv", parse_dates=["timestamp"])
    df_as = pd.read_csv("analytics_logs.csv", parse_dates=["timestamp"])

    # Filter logs by the time window
    df_dd = df_dd[(df_dd["timestamp"] >= start_time) & (df_dd["timestamp"] <= end_time)]
    df_be = df_be[(df_be["timestamp"] >= start_time) & (df_be["timestamp"] <= end_time)]
    df_as = df_as[(df_as["timestamp"] >= start_time) & (df_as["timestamp"] <= end_time)]

    # Consolidate logs by merging on session_id, event_idx, and event_name.
    # This join will include BE logs only for events that have a backend call (i.e. where a correlation_id exists in DD).
    df_consolidated = pd.merge(
        df_dd,
        df_be,
        on=["session_id", "event_idx", "event_name"],
        how="left",
        suffixes=("_dd", "_be")
    )

    # Now merge the Analytics logs (which don't include correlation_id) on session_id, event_idx, and event_name.
    df_consolidated = pd.merge(
        df_consolidated,
        df_as,
        on=["session_id", "event_idx", "event_name"],
        how="left"
    )

    # Optional: sort the consolidated DataFrame by session_id and event_idx
    df_consolidated.sort_values(by=["session_id", "event_idx"], inplace=True)

    # Display a sample of the consolidated snapshot
    print("Consolidated Logs Snapshot:")
    print(df_consolidated.head(10))
    # Save the consolidated view to CSV if desired
    df_consolidated.to_csv("consolidated_logs.csv", index=False)
    df = df_consolidated

In [2]:
 # Define a time window for consolidation (adjust as needed)
# 2025-01-17 09:38:37 - 2025-01-17 11:10:29
start_time = pd.to_datetime("2025-01-17 09:38:37")
end_time   = pd.to_datetime("2025-01-17 11:10:29")
consolidate(start_time, end_time)

Consolidated Logs Snapshot:
         timestamp_dd   session_id  event_idx          event_name platform   
0 2025-01-17 10:19:09  sess-A10320          1              Splash  Android  \
1 2025-01-17 10:19:56  sess-A10320          2           LoginPage  Android   
2 2025-01-17 10:20:06  sess-A10320          3        LoginAttempt  Android   
3 2025-01-17 10:20:19  sess-A10320          4  LoginResultFailure  Android   
4 2025-01-17 10:21:01  sess-A10320          5                Exit  Android   
5 2025-01-17 10:30:40  sess-A10461          1              Splash  Android   
6 2025-01-17 10:31:34  sess-A10461          2                Home  Android   
7 2025-01-17 10:31:45  sess-A10461          3         ArticleText  Android   
8 2025-01-17 10:32:36  sess-A10461          4               Share  Android   
9 2025-01-17 10:33:21  sess-A10461          5         ArticleText  Android   

                                log_message_dd article_id_dd   
0               SplashScreenActivity displayed   

In [50]:
# Find a session with session_id = sess-I14083
dd_df = pd.read_csv("datadog_mobile_logs.csv")
dd_df.head()

filtered_df = dd_df[dd_df["session_id"] == "sess-I14083"]
filtered_df.reset_index(inplace=True)
filtered_df



,index,timestamp,session_id,event_idx,event_name,platform,log_message,article_id,correlation_id
0,34460,2025-01-21 23:08:33,sess-I14083,1,Splash,iOS,SplashScreenViewController displayed,NaN,NaN
1,34461,2025-01-21 23:08:58,sess-I14083,2,Home,iOS,HomeScreenViewController displayed,NaN,CORR-sess-I14083-2
2,34462,2025-01-21 23:09:49,sess-I14083,3,ArticleVideo,iOS,ArticleVideoViewController opened,nyt53472165,CORR-sess-I14083-3
3,34463,2025-01-21 23:09:56,sess-I14083,4,Share,iOS,ShareViewController: User clicked share,NaN,CORR-sess-I14083-4
4,34464,2025-01-21 23:10:29,sess-I14083,5,ArticleVideo,iOS,ArticleVideoViewController opened,nyt73433694,CORR-sess-I14083-5
5,34465,2025-01-21 23:10:59,sess-I14083,6,AudioArticleError,iOS,ArticleErrorActivity: Audio outage – AI TTS se...,cnn79789763,CORR-sess-I14083-6
6,34466,2025-01-21 23:11:29,sess-I14083,7,Share,iOS,ShareViewController: User clicked share,NaN,CORR-sess-I14083-7
7,34467,2025-01-21 23:12:04,sess-I14083,8,Exit,iOS,User exited app from LastScreenViewController,NaN,NaN
